In [ ]:
#workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
import os
This_module_path = os.getcwd()  #os.path.dirname(os.path.abspath(__file__))
workdir_ = os.path.join(This_module_path, '../../' )
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")

print( f" In {__name__} we have This_module_path={This_module_path} " )
print( f" In {__name__} we have workdir_={workdir_} " )
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
sys.path.append(workdir_ + 'myPythonTools/Plotting/')
sys.path.append(workdir_ + 'myPythonTools/CASutils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import utils as uti
import numerical_utils as nuti
import validation_data as Val
import var_A_x_B as vAB
import MakePressures as MkP
import GridUtils as GrU
import LatLonMaps as LL
import filter_utils as fu

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( LL )
importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)


In [ ]:
exp='POLARRES_FMT_x01_F'
#exp='fmthist_MM_control_TEM_2'
B = uti.MakeDict4Exp( exp=exp , user='juliob', subd='regridded' , hsPat='cam.h1' , ymdPat='1994-07-*' ,verbose=True, open_dataset=True )

Q = uti.MakeDict4Exp( exp=exp , user='juliob', subd='regridded' , hsPat='cam.h0' , ymdPat='1994-07*' ,verbose=True, open_dataset=True )
phis=Q.X.PHIS.values/9.8

In [ ]:
"""
lon0,lat0=100.,-50.
zoog = np.where( (np.abs(lat_x - lat0)<1)&(np.abs(lon_x - lon0)<1) )
print(zoog[0])
"""

In [ ]:
########################
# Some massaging
########################

hyai_x=B.X.hyai.values
hybi_x=B.X.hybi.values
hyam_x=B.X.hyam.values
hybm_x=B.X.hybm.values

#ps_x =B.X.PS.values
lon_x=B.X.lon.values
lat_x=B.X.lat.values
lev_x=B.X.lev.values
zlev_x=-7.0*np.log( lev_x/1_000. )


In [ ]:

print(zlev_x[40])
print(zlev_x[80])


In [ ]:
og18=B.X.OMEGA[:, 40, :].values
u01=B.X.U[:, 80, :].values
v01=B.X.V[:, 80, :].values
#data2=B.X.T.values
shf=B.X.SHFLX.values

In [ ]:
print( np.shape(u01))
nt,ny,nx = np.shape(u01)
zeta01 = np.zeros( (nt,ny,nx) )

for t in np.arange( nt ):
    if ( (t%10)==0):
        print( f'time {t}' ) 
    zeta01[t,:,:] = nuti.Sphere_Curl2( f_x=u01[t,:,:], f_y=v01[t,:,:] , lat=lat_x, lon=lon_x, wrap=True  ) 

In [ ]:
plt.contourf( zeta01[0,:,:] )

In [ ]:
omega_gridKey = GrU.gridKey( B.X.OMEGA )
print( omega_gridKey )


In [ ]:
#pdata = data[ : , 40 , : ]
#pdata2 = data2 #[ :, -1 , :] - data2[ :, -10, : ]

In [ ]:
# Only possible on lat-lon field

nt,nz,ny,nx = np.shape( B.X.OMEGA )
ss=np.zeros((nt,ny,nx) )
for t in np.arange( nt ):
    if ( (t%10)==0):
        print( f'time {t}' ) 
    ss[t,:,:]=fu.filterk( B.X.OMEGA[t,40,:,:] , kmin=0,kmax=20,dimlon=1 )

In [ ]:
pdata = pdata - ss

In [ ]:
#doodoo
print( f"{B.X.time[0].dt.strftime('%Y-%m-%d %HZ').values}" )#.values )
print( len(B.X.time) )

In [ ]:

levels=np.linspace(-.1, .1, num=21)
levels2=np.linspace( -100,100,num=11)

fig, ax = plt.subplots(figsize=(20,10))
t=90
#contour = ax.contourf(lon_x, lat_x, ss[0, :,:], levels=levels, cmap='bwr')
if ( omega_gridKey == 'tzyx' ):
    contour = ax.contourf(lon_x, lat_x, pdata[t, :,:], levels=levels, cmap='bwr')
    #contour2 = ax.contour(lon_x, lat_x, pdata2[t,:,:], levels=levels2, alpha=0.5,cmap='plasma')  # Add a second contour plot for data2
    land = ax.contour( lon_x, lat_x, phis[0,:,:],levels=[0,1,1000],colors='black')
elif ( omega_gridKey == 'tzc' ):
    contour = ax.tricontourf(lon_x, lat_x, pdata[t, : ], levels=levels, cmap='bwr')
    contour2 = ax.tricontour(lon_x, lat_x, pdata2[t,:], levels=levels2, alpha=0.5,cmap='plasma')  # Add a second contour plot for data2
    land = ax.tricontour( lon_x, lat_x, phis[0,:],levels=[0,1,1000],colors='black')
annot= ax.text( 10,-84,B.X.time[t].dt.strftime('%Y-%m-%d %HZ').values , fontsize=16, bbox=dict(facecolor='white') )
cbar = plt.colorbar(contour, ax=ax, shrink=0.6)  # Create colorbar
#cbar2 = plt.colorbar(contour2, ax=ax, shrink=0.6)  # Create colorbar



In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

nframes = len(B.X.time) 
#levels=0.2*np.linspace(-.2, .2, num=21)
#levels=np.linspace(-.2, .2, num=21)
#levels2=np.linspace( 240,300,num=11)
levels=np.linspace(-.1, .1, num=21)
levels2=np.linspace( -100,100,num=11)

# Create the figure and axis
# levels=0.001*np.linspace(-200, 200, num=21)
fig, ax = plt.subplots(figsize=(20,10))
t=0
#contour = ax.contourf(lon_x, lat_x, ss[0, :,:], levels=levels, cmap='bwr')
if ( omega_gridKey == 'tzyx' ):
    contour = ax.contourf(lon_x, lat_x, pdata[t, :,:], levels=levels, cmap='bwr')
    contour2 = ax.contour(lon_x, lat_x, pdata2[t,:,:], levels=levels2, alpha=0.5,cmap='plasma')  # Add a second contour plot for data2
    land = ax.contour( lon_x, lat_x, phis[0,:,:],levels=[0,1,1000],colors='black')
elif ( omega_gridKey == 'tzc' ):
    contour = ax.tricontourf(lon_x, lat_x, pdata[t, : ], levels=levels, cmap='bwr')
    contour2 = ax.tricontour(lon_x, lat_x, pdata2[t,:], levels=levels2, alpha=0.5,cmap='plasma')  # Add a second contour plot for data2
    land = ax.tricontour( lon_x, lat_x, phis[0,:],levels=[0,1,1000],colors='black')
annot= ax.text( 10,-84,B.X.time[t].dt.strftime('%Y-%m-%d %HZ').values , fontsize=16, bbox=dict(facecolor='white') )
cbar = plt.colorbar(contour, ax=ax, shrink=0.6)  # Create colorbar


# Update function for the animation
def update(frame):
    global contour,contour2
    for coll in contour.collections:
        coll.remove()  # Remove existing contour collections
    for coll in contour2.collections:
        coll.remove()  # Remove existing contour collections
    if ( omega_gridKey == 'tzyx' ):
        contour = ax.contourf(lon_x, lat_x, pdata[frame, :,:], levels=levels, cmap='bwr')
        contour2 = ax.contour(lon_x, lat_x, pdata2[frame,:,:], levels=levels2, alpha=0.5,cmap='plasma')  # Add a second contour plot for data2
        land = ax.contour( lon_x, lat_x, phis[0,:,:],levels=[0,1,1000],colors='black')
    if ( omega_gridKey == 'tzc' ):
        contour = ax.tricontourf(lon_x, lat_x, pdata[frame, :] , levels=levels, cmap='bwr')
        contour2 = ax.tricontour(lon_x, lat_x, pdata2[frame,:], levels=levels2, alpha=0.5,cmap='plasma')  # Add a second contour plot for data2
        land = ax.tricontour( lon_x, lat_x, phis[0,:],levels=[0,1,1000],colors='black')
    annot= ax.text( 10,-84,B.X.time[frame].dt.strftime('%Y-%m-%d %HZ').values , fontsize=16, bbox=dict(facecolor='white') )
    cbar.update_normal(contour)  # Update colorbar
    #cbar = plt.colorbar(contour, shrink=.6)
    return contour.collections+contour2.collections

# Create and display the animation
ani = FuncAnimation(fig, update, frames=nframes, interval=50, blit=True)
plt.close()  # Prevents the initial empty plot from showing up
HTML(ani.to_html5_video())



In [ ]:
# Step 7: Save the animation as HTML5 video
ani.save('../AnimationFiles/OMEGA-18km-SHFLX-Jul-1994-POLARRES-25km-20240319.mp4', writer='ffmpeg')

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

levels=np.linspace(-.2, .2, num=21)
levels2=np.linspace( 240,300,num=11)


# Create the figure and axis
# levels=0.001*np.linspace(-200, 200, num=21)
fig, ax = plt.subplots(figsize=(20,10))
contour = ax.contourf(lon_x, lat_x, data[0, 40, :,:], levels=levels, cmap='bwr')
contour2 = ax.contour(lon_x, lat_x, data2[0,90,:,:], levels=levels2, alpha=0.5,cmap='cool')  # Add a second contour plot for data2
land = ax.contour( lon_x, lat_x, phis[0,:,:],levels=[0,1,1000],colors='black')
cbar = plt.colorbar(contour, ax=ax, shrink=0.6)  # Create colorbar

# Update function for the animation
def update(frame):
    global contour,contour2
    for coll in contour.collections:
        coll.remove()  # Remove existing contour collections
    for coll in contour2.collections:
        coll.remove()  # Remove existing contour collections
    contour = ax.contourf(lon_x, lat_x, data[frame, 40, :,:], levels=levels, cmap='bwr')
    contour2 = ax.contour(lon_x, lat_x, data2[frame,90,:,:], levels=levels2, alpha=0.5,cmap='cool')  # Add a second contour plot for data2
    land = ax.contour( lon_x, lat_x, phis[0,:,:],levels=[0,1,1000],colors='black')
    cbar.update_normal(contour)  # Update colorbar
    #cbar = plt.colorbar(contour, shrink=.6)
    return contour.collections+contour2.collections

# Create and display the animation
ani = FuncAnimation(fig, update, frames=120, interval=50, blit=True)
plt.close()  # Prevents the initial empty plot from showing up
HTML(ani.to_html5_video())


In [ ]:

levels=21 #0.001*np.linspace(-200, 200, num=21)
levels=np.linspace(-.2, .2, num=21)

#plt.tricontourf( lon_x, lat_x, data[0,92,:],levels=levels)
#plt.contourf( lon_x, lat_x, data[0,40,:,:],levels=levels)
#plt.contourf( lon_x, lat_x, phis[0,:,:],levels=[0,1,10,1000])

fig, ax = plt.subplots()
omega_con = ax.contourf(lon_x, lat_x, data[0, 40, :,:], levels=levels, cmap='viridis')
phis_con=ax.contourf( lon_x, lat_x, phis[0,:,:],levels=[0,1,10,1000])
cbar = plt.colorbar(omega_con, ax=ax, shrink=0.6)  # Create colorbar


In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML



# Create the figure and axis
# levels=0.001*np.linspace(-200, 200, num=21)
fig, ax = plt.subplots(figsize=(20,10))
contour = ax.contourf(lon_x, lat_x, data[0, 40, :,:], levels=levels, cmap='bwr')
land = ax.contour( lon_x, lat_x, phis[0,:,:],levels=[0,1,1000],colors='black')
cbar = plt.colorbar(contour, ax=ax, shrink=0.6)  # Create colorbar

# Update function for the animation
def update(frame):
    global contour
    for coll in contour.collections:
        coll.remove()  # Remove existing contour collections
    contour = ax.contourf(lon_x, lat_x, data[frame, 40, :,:], levels=levels, cmap='bwr')
    land = ax.contour( lon_x, lat_x, phis[0,:,:],levels=[0,1,1000],colors='black')
    cbar.update_normal(contour)  # Update colorbar
    #cbar = plt.colorbar(contour, shrink=.6)
    return contour.collections

# Create and display the animation
ani = FuncAnimation(fig, update, frames=120, interval=50, blit=True)
plt.close()  # Prevents the initial empty plot from showing up
HTML(ani.to_html5_video())

In [ ]:
# Create the figure and axis
levels=np.linspace(-.2, .2, num=21)
levels2=np.linspace( 240,280,num=11)
fig, ax = plt.subplots(figsize=(20,10))
contour = ax.contourf(lon_x, lat_x, data[0, 40, :,:], levels=levels, cmap='bwr')
contour2 = ax.contour(lon_x, lat_x, data2[0,90,:,:], levels=levels2, alpha=0.5,cmap='cool')  # Add a second contour plot for data2
land = ax.contour(lon_x, lat_x, phis[0,:,:], levels=[0,1,1000],colors='black')
cbar = plt.colorbar(contour, ax=ax, shrink=0.6)  # Create colorbar for the first contour plot
#cbar2 = plt.colorbar(contour2, ax=ax, shrink=0.6)  # Create colorbar for the second contour plot


In [ ]:
# Create the figure and axis
levels=np.linspace(-.2, .2, num=21)
levels2=np.linspace( 220,280,num=11)
fig, ax = plt.subplots() #(figsize=(20,10))
contour = ax.contourf(lon_x, lat_x, data[0, 40, :,:], levels=levels, cmap='bwr')
contour2 = ax.contour(lon_x, lat_x, data2[0,90,:,:], levels=levels2, alpha=0.5,cmap='cool')  # Add a second contour plot for data2
land = ax.contour(lon_x, lat_x, phis[0,:,:], levels=[0,1,1000],colors='black')
cbar = plt.colorbar(contour, ax=ax, shrink=0.6)  # Create colorbar for the first contour plot
#cbar2 = plt.colorbar(contour2, ax=ax, shrink=0.6)  # Create colorbar for the second contour plot
# Update function for the animation
def update(frame):
    global contour, contour2
    for coll in contour.collections:
        coll.remove()  # Remove existing contour collections for the first data variable
    for coll in contour2.collections:
        coll.remove()  # Remove existing contour collections for the second data variable
    contour = ax.contourf(lon_x, lat_x, data[frame, 40, :,:], levels=levels, cmap='bwr')
    contour2 = ax.contourf(lon_x, lat_x, data2[frame, 40, :,:], levels=levels2, cmap='cool', alpha=0.5)  # Update the second contour plot
    land = ax.contour(lon_x, lat_x, phis[0,:,:], levels=[0,1,10,1000])
    cbar.update_normal(contour)  # Update colorbar for the first contour plot
    #cbar2.update_normal(contour2)  # Update colorbar for the second contour plot
    return contour.collections + contour2.collections

# Create and display the animation
ani = FuncAnimation(fig, update, frames=120, interval=50, blit=True)


In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML



# Create the figure and axis
# levels=0.001*np.linspace(-200, 200, num=21)
fig, ax = plt.subplots()
contour = ax.tricontourf(lon_x, lat_x, data[0, 92, :], levels=levels, cmap='viridis')
cbar = plt.colorbar(contour, ax=ax, shrink=0.6)  # Create colorbar

# Update function for the animation
def update(frame):
    global contour
    for coll in contour.collections:
        coll.remove()  # Remove existing contour collections
    contour = ax.tricontourf(lon_x, lat_x, data[frame, 92, :], levels=levels, cmap='viridis')
    cbar.update_normal(contour)  # Update colorbar
    #cbar = plt.colorbar(contour, shrink=.6)
    return contour.collections

# Create and display the animation
ani = FuncAnimation(fig, update, frames=120, interval=50, blit=True)
plt.close()  # Prevents the initial empty plot from showing up
HTML(ani.to_html5_video())

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML



# Create the figure and axis
# levels=0.001*np.linspace(-200, 200, num=21)
fig, ax = plt.subplots()
contour = ax.tricontourf(lon_x, lat_x, data[0, 92, :], levels=levels, cmap='viridis')
cbar = plt.colorbar(contour, ax=ax, shrink=0.6)  # Create colorbar

# Update function for the animation
def update(frame):
    global contour
    for coll in contour.collections:
        coll.remove()  # Remove existing contour collections
    contour = ax.tricontourf(lon_x, lat_x, data[frame, 92, :], levels=levels, cmap='viridis')
    cbar.update_normal(contour)  # Update colorbar
    #cbar = plt.colorbar(contour, shrink=.6)
    return contour.collections

# Create and display the animation
ani = FuncAnimation(fig, update, frames=120, interval=50, blit=True)
plt.close()  # Prevents the initial empty plot from showing up
HTML(ani.to_html5_video())

In [ ]:
# Step 7: Save the animation as HTML5 video
ani.save('animation-wpthlp-Sep-2001.mp4', writer='ffmpeg')

In [ ]:
import math
result = math.ceil(22 / 3)
print(result)  # 